In [4]:
#%pip install transformers --upgrade

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
#%pip install torch --upgrade

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [46]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [47]:
def load_and_prepare(path):
    df = pd.read_csv(path, on_bad_lines='skip')
    df['label'] = df['Оценка'].apply(lambda x: 0 if x < 3 else (1 if x == 3 else 2))
    df = df.dropna(subset=['Отзыв','Оценка'])
    return df

In [48]:
df = load_and_prepare('yandex_reviews_main_last_right_version.csv')

In [49]:
df

,Название точки,Ссылка,Отзыв,Оценка,Тип организации,label
0,Медицинский центр доктора Болдырева,https://yandex.ru/maps/org/meditsinskiy_tsentr...,"Этот медицинский центр, лучшее, что мы видели ...",5.0,Детская поликлиника,2
1,Медицинский центр доктора Болдырева,https://yandex.ru/maps/org/meditsinskiy_tsentr...,"Очень приятные, вежливые и предусмотрительные ...",5.0,Детская поликлиника,2
2,Медицинский центр доктора Болдырева,https://yandex.ru/maps/org/meditsinskiy_tsentr...,"Редкий, крутейший и очень ответственный специа...",5.0,Детская поликлиника,2
3,Медицинский центр доктора Болдырева,https://yandex.ru/maps/org/meditsinskiy_tsentr...,Хочу выразить свою благодарность Болдыреву Вла...,5.0,Детская поликлиника,2
4,Медицинский центр доктора Болдырева,https://yandex.ru/maps/org/meditsinskiy_tsentr...,Уже более пяти лет посещаем врача Владимира Ге...,5.0,Детская поликлиника,2
...,...,...,...,...,...,...
106011,ВС РФ платные медицинские услуги Главный военн...,https://yandex.ru/maps/org/vs_rf_platnyye_medi...,Советую посещать данное заведение!,5.0,Госпиталь,2
106012,ВС РФ платные медицинские услуги Главный военн...,https://yandex.ru/maps/org/vs_rf_platnyye_medi...,"Отличные врачи, новое оборудование. Спасибо за...",5.0,Госпиталь,2
106013,ВС РФ платные медицинские услуги Главный военн...,https://yandex.ru/maps/org/vs_rf_platnyye_medi...,Скоро прийдёт новый начальник и надеюсь постав...,3.0,Госпиталь,1
106014,ВС РФ платные медицинские услуги Главный военн...,https://yandex.ru/maps/org/vs_rf_platnyye_medi...,Отличный госпиталь.,5.0,Госпиталь,2


In [50]:
import re
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  
    text = re.sub(r'[^\w\s.,!?]', '', text)
    return text.strip()

df['Отзыв'] = df['Отзыв'].apply(clean_text)

In [51]:
train_df, temp_df = train_test_split(
    df, test_size=0.3, stratify=df['label'], random_state=42
)
val_df, test_df = train_test_split(
    temp_df, test_size=0.25, stratify=temp_df['label'], random_state=42
)


In [52]:
class_weights = compute_class_weight(
    'balanced', classes=np.unique(train_df['label']), y=train_df['label']
)
class_weights = torch.tensor(class_weights, dtype=torch.float)

In [53]:
class_weights

tensor([2.0521, 6.8099, 0.4227])

In [54]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = 'deepvk/roberta-base'

tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DataCollatorWithPadding(tokenizer)
max_length = 256

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at deepvk/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [55]:
import warnings
warnings.filterwarnings("ignore")

class ReviewDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.encodings = tokenizer(
            texts.tolist(),
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        self.labels = torch.tensor(labels.tolist(), dtype=torch.long)

    def __len__(self): return len(self.labels)

    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

In [56]:
train_ds = ReviewDataset(train_df['Отзыв'], train_df['label'], tokenizer, max_length)
val_ds   = ReviewDataset(val_df['Отзыв'], val_df['label'], tokenizer, max_length)
test_ds  = ReviewDataset(test_df['Отзыв'], test_df['label'], tokenizer, max_length)

In [57]:
def compute_loss_with_weights(self, model, inputs, return_outputs=False):
    labels = inputs.pop('labels')
    outputs = model(**inputs)
    logits = outputs.logits
    loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))
    loss = loss_fct(logits, labels)
    return (loss, outputs) if return_outputs else loss

type(model).compute_loss = compute_loss_with_weights

In [58]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds  = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted'
    )
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}


In [59]:
training_args = TrainingArguments(
    output_dir='./out',
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    #logging_dir='./logs',
    #logging_steps=100,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    learning_rate=3e-5,
    save_total_limit=2
)

In [17]:
#%pip install -U accelerate --upgrade

In [60]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [61]:
trainer.train()

 22%|██▏       | 500/2316 [03:17<11:46,  2.57it/s]

{'loss': 0.2583, 'grad_norm': 1.1234705448150635, 'learning_rate': 2.3536269430051816e-05, 'epoch': 0.86}


 99%|█████████▉| 186/187 [00:25<00:00,  7.63it/s]
                                                  
 25%|██▌       | 580/2316 [04:14<09:05,  3.18it/s][A
                                                 

{'eval_loss': 0.20438310503959656, 'eval_accuracy': 0.9265211917750734, 'eval_f1': 0.9110957180932658, 'eval_precision': 0.9080269872658079, 'eval_recall': 0.9265211917750734, 'eval_runtime': 25.5721, 'eval_samples_per_second': 931.874, 'eval_steps_per_second': 7.313, 'epoch': 1.0}


 43%|████▎     | 1000/2316 [07:26<08:32,  2.57it/s] 

...

 99%|█████████▉| 186/187 [00:25<00:00,  7.64it/s]
                                                   
100%|██████████| 2316/2316 [18:14<00:00,  2.57it/s]A
                                                 

{'eval_loss': 0.22261764109134674, 'eval_accuracy': 0.9282836760386068, 'eval_f1': 0.9271276619839521, 'eval_precision': 0.9260551493780307, 'eval_recall': 0.9282836760386068, 'eval_runtime': 25.3444, 'eval_samples_per_second': 940.246, 'eval_steps_per_second': 7.378, 'epoch': 3.99}


100%|██████████| 2316/2316 [18:40<00:00,  2.07it/s]

{'train_runtime': 1120.7585, 'train_samples_per_second': 264.603, 'train_steps_per_second': 2.066, 'train_loss': 0.17492220191757912, 'epoch': 3.99}


TrainOutput(global_step=2316, training_loss=0.17492220191757912, metrics={'train_runtime': 1120.7585, 'train_samples_per_second': 264.603, 'train_steps_per_second': 2.066, 'total_flos': 3.895986154652928e+16, 'train_loss': 0.17492220191757912, 'epoch': 3.993960310612597})

In [62]:
metrics = trainer.evaluate(test_ds)
print("Test metrics:", metrics)

100%|██████████| 63/63 [00:08<00:00,  7.44it/s]

Test metrics: {'eval_loss': 0.2293693572282791, 'eval_accuracy': 0.9277442094662638, 'eval_f1': 0.9263552996727994, 'eval_precision': 0.9250716259586114, 'eval_recall': 0.9277442094662638, 'eval_runtime': 8.6287, 'eval_samples_per_second': 920.649, 'eval_steps_per_second': 7.301, 'epoch': 3.993960310612597}


In [63]:
from torch.utils.data import DataLoader, TensorDataset

In [64]:
texts = df['Отзыв'].tolist()
enc = tokenizer(texts, max_length=max_length, padding='max_length',
                truncation=True, return_tensors='pt')
dataset = TensorDataset(enc['input_ids'], enc['attention_mask'])

loader = DataLoader(dataset, batch_size=32)

model.eval()
all_probs = []
with torch.no_grad():
    for input_ids, attention_mask in loader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
        probs = torch.nn.functional.softmax(logits, dim=1).cpu()
        all_probs.append(probs)

probs = torch.cat(all_probs, dim=0).numpy()

In [65]:
import os


output_dir = './model_save/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)



Saving model to ./model_save/


('./model_save/tokenizer_config.json',
 './model_save/special_tokens_map.json',
 './model_save/vocab.json',
 './model_save/merges.txt',
 './model_save/added_tokens.json',
 './model_save/tokenizer.json')

In [7]:
#from transformers import AutoTokenizer, AutoModelForSequenceClassification


#output_dir = './out/checkpoint-2316/'
#model_name = 'DeepPavlov/xlm-roberta-large-en-ru'
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


#model = AutoModelForSequenceClassification.from_pretrained(output_dir)
#tokenizer = AutoTokenizer.from_pretrained(output_dir)

#model.to(device)

#max_length = 256



In [66]:
import pandas as pd
import numpy as np


probs_df = pd.DataFrame(
    probs,
    columns=['p_neg', 'p_neu', 'p_pos']
)
probs_df.index = df.index

probs_df['company'] = df['Название точки']

# нормированная разность
probs_df['norm_diff'] = (probs_df['p_pos'] - probs_df['p_neg']) / (
    probs_df['p_pos'] + probs_df['p_neg'] + 1e-12
)
# взвешенный индекс
probs_df['weighted_score'] = (
    0 * probs_df['p_neg'] +
    0.5 * probs_df['p_neu'] +
    1 * probs_df['p_pos']
)
# доля положительных среди ненейтральных
probs_df['pos_over_non_neu'] = probs_df['p_pos'] / (
    probs_df['p_pos'] + probs_df['p_neg'] + 1e-12
)

agg = probs_df.groupby('company').agg(
    avg_p_positive        = ('p_pos', 'mean'),
    avg_norm_diff         = ('norm_diff', 'mean'),
    avg_weighted_score    = ('weighted_score', 'mean'),
    avg_pos_over_non_neu  = ('pos_over_non_neu', 'mean'),
    n_reviews             = ('p_pos', 'size')  # на всякий случай
).reset_index()

In [67]:
agg

,company,avg_p_positive,avg_norm_diff,avg_weighted_score,avg_pos_over_non_neu,n_reviews
0,3D Medica,0.925512,0.902389,0.942740,0.951194,26
1,9 Лечебно-диагностический центр Минобороны России,0.902196,0.819989,0.911512,0.909994,210
2,9 Лечебно-диагностический центр Минобороны Рос...,0.877916,0.791890,0.897115,0.895945,599
3,ABC-медицина,0.828762,0.686843,0.844019,0.843421,715
4,Advanced Fertility Clinic,0.736681,0.506929,0.748898,0.753464,37
...,...,...,...,...,...,...
273,Центральный военный клинический госпиталь имен...,0.923874,0.874973,0.934978,0.937486,183
274,Эксимер,0.753100,0.549699,0.779702,0.774849,506
275,ЭммаКлиник,0.707634,0.420279,0.721097,0.710140,48
276,Эстетическая клиника Век Адалин,0.908881,0.838854,0.917075,0.919427,155


In [68]:
agg.to_csv('companies_ethics_indexes_deepvk.csv',index=False)